In [ ]:

import pandas as pd
import numpy as np
import networkx as nx
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

Movies

In [ ]:
movies = pd.read_csv("/content/movies_metadata.csv")
print(movies.columns)
movies.head()

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
movies['genres'] = movies['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
keywords = pd.read_csv('/content/keywords.csv')


In [ ]:
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

movies['id'] = movies['id'].apply(clean_ids)
movies = movies[movies['id'].notnull()]

In [ ]:
movies['id'] = movies['id'].astype('int')
keywords['id'] = keywords['id'].astype('int')

movies = movies.merge(keywords, on='id')

movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
movies["keywords"] = movies["keywords"].apply(literal_eval)

In [ ]:
def generate_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 10:
            names = names[:10]
        return names
    return []

movies['keywords'] = movies['keywords'].apply(generate_list)
movies['genres'] = movies['genres'].apply(lambda x: x[:10])

movies[['title', 'keywords', 'genres']].head()

,title,keywords,genres
0,Toy Story,"[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]"
1,Jumanji,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[fishing, best friend, duringcreditsstinger, o...","[Romance, Comedy]"
3,Waiting to Exhale,"[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]"
4,Father of the Bride Part II,"[baby, midlife crisis, confidence, aging, daug...",[Comedy]


In [ ]:
def sanitize(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ','')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ', ''))
        else:
            return ''

for feature in ['genres', 'keywords']:
    movies[feature] = movies[feature].apply(sanitize)

In [ ]:
movies[['title', 'keywords', 'genres']].head()

,title,keywords,genres
0,Toy Story,"[jealousy, toy, boy, friendship, friends, riva...","[animation, comedy, family]"
1,Jumanji,"[boardgame, disappearance, basedonchildren'sbo...","[adventure, fantasy, family]"
2,Grumpier Old Men,"[fishing, bestfriend, duringcreditsstinger, ol...","[romance, comedy]"
3,Waiting to Exhale,"[basedonnovel, interracialrelationship, single...","[comedy, drama, romance]"
4,Father of the Bride Part II,"[baby, midlifecrisis, confidence, aging, daugh...",[comedy]


In [ ]:
def movie_soup(x):
    return  x["title"] + " " + " ".join(x['genres']) + " "+x['overview']+" "+" ".join(x['keywords'])

movies['overview'] = movies['overview'].fillna('')
movies['title'] = movies['title'].fillna('')
movies['soup'] = movies.apply(movie_soup, axis=1)

In [ ]:
movies.loc[movies['title']=="The Matrix",'soup'].values

array(['The Matrix action sciencefiction Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth. savingtheworld artificialintelligence manvsmachine philosophy prophecy martialarts selfsacrifice fight insurgence virtualreality'],
      dtype=object)

In [ ]:
movies2=movies.iloc[:200]
movies2

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords,soup
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[animation, comedy, family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[jealousy, toy, boy, friendship, friends, riva...",Toy Story animation comedy family Led by Woody...
1,False,NaN,65000000,"[adventure, fantasy, family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[boardgame, disappearance, basedonchildren'sbo...",Jumanji adventure fantasy family When siblings...
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[romance, comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[fishing, bestfriend, duringcreditsstinger, ol...",Grumpier Old Men romance comedy A family weddi...
3,False,NaN,16000000,"[comedy, drama, romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[basedonnovel, interracialrelationship, single...",Waiting to Exhale comedy drama romance Cheated...
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[baby, midlifecrisis, confidence, aging, daugh...",Father of the Bride Part II comedy Just when G...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,False,NaN,42000000,"[crime, drama, sciencefiction, thriller]",http://www.strange-days.com,281,tt0114558,en,Strange Days,Set in the year 1999 during the last days of t...,...,145.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,You know you want it.,Strange Days,False,6.8,334.0,"[pornography, prostitute, rape, policebrutalit...",Strange Days crime drama sciencefiction thrill...
196,False,NaN,0,"[drama, romance]",NaN,5967,tt0058450,fr,Les parapluies de Cherbourg,This simple romantic tragedy begins in 1957. G...,...,91.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,a film for all the young lovers of the world,The Umbrellas of Cherbourg,False,7.6,119.0,"[carmechanic, letter, musical, war, single]",The Umbrellas of Cherbourg drama romance This ...
197,False,NaN,14000000,[thriller],NaN,79593,tt0114666,en,The Tie That Binds,John Netherwood and his wife Leann are fugitiv...,...,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Tie That Binds,False,4.6,9.0,[],The Tie That Binds thriller John Netherwood an...
198,False,NaN,0,"[drama, fantasy]",NaN,47939,tt0114663,en,Three Wishes,While Jane Holman (Mastrantonio) is driving wi...,...,115.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"If you really believe, magic will find you",Three Wishes,False,6.1,18.0,"[baseball, drifter, dog, koreanwar, womandirec...",Three Wishes drama fantasy While Jane Holman (...


In [ ]:
movsoup= movies['soup']

In [ ]:
count = CountVectorizer(stop_words = "english")
count.fit(movsoup)

CountVectorizer(stop_words='english')

In [ ]:
movies_matrix = count.transform(movies2['soup'])

movies_matrix.shape

(200, 91312)

In [ ]:
cosine_sim_movies = cosine_similarity(movies_matrix, movies_matrix)

print(cosine_sim_movies.shape)
print(cosine_sim_movies)

(200, 200)
[[1.         0.03344968 0.03698001 ... 0.         0.06249751 0.        ]
 [0.03344968 1.         0.06030227 ... 0.         0.02911798 0.        ]
 [0.03698001 0.06030227 1.         ... 0.03849002 0.02146076 0.05443311]
 ...
 [0.         0.         0.03849002 ... 1.         0.         0.04714045]
 [0.06249751 0.02911798 0.02146076 ... 0.         1.         0.03942593]
 [0.         0.         0.05443311 ... 0.04714045 0.03942593 1.        ]]


In [ ]:
G1_w = nx.from_numpy_matrix(np.matrix(cosine_sim_movies), create_using=nx.Graph)
layout = nx.spring_layout(G1_w)
# plt.figure(figsize=(16,10))

# nx.draw(G1_w, layout)
# labels = nx.get_edge_attributes(G1_w, "weight")
# nx.draw_networkx_edge_labels(G1_w, pos=layout, edge_labels=labels)
# # nx.draw_networkx_edge_labels(G, pos=layout)
# plt.show()

ROC reciever operator charecteristic AUC- area under curve is greater-> better model

In [ ]:
for i in range(200):
  for j in range(200):
    if (cosine_sim_movies[i][j]>=0.05):
      cosine_sim_movies[i][j]=1
    else :
      cosine_sim_movies[i][j]=0
print(cosine_sim_movies)


[[1. 0. 0. ... 0. 1. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 1.]]


In [ ]:
import pandas as pd
import networkx as nx
df = pd.DataFrame(cosine_sim_movies)
G1_uw = nx.from_pandas_adjacency(df)
print(G1_uw)

Graph with 200 nodes and 4596 edges


In [ ]:
# plt.figure(figsize=(16,10))
# nx.draw(G1_uw, node_size=2500, with_labels=True, pos=nx.kamada_kawai_layout(G))

Books

In [ ]:
books = pd.read_csv("/content/data.csv")
print(books.columns)
books.head()


Index(['index', 'title', 'genre', 'summary'], dtype='object')


,index,title,genre,summary
0,0,Drowned Wednesday,fantasy,Drowned Wednesday is the first Trustee among ...
1,1,The Lost Hero,fantasy,"As the book opens, Jason awakens on a school ..."
2,2,The Eyes of the Overworld,fantasy,Cugel is easily persuaded by the merchant Fia...
3,3,Magic's Promise,fantasy,The book opens with Herald-Mage Vanyel return...
4,4,Taran Wanderer,fantasy,Taran and Gurgi have returned to Caer Dallben...


In [ ]:
def sanitize(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ','')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ', ''))
        else:
            return ''

for feature in ['genre']:
    books[feature] = books[feature].apply(sanitize)

In [ ]:
books.head()

,index,title,genre,summary
0,0,Drowned Wednesday,fantasy,Drowned Wednesday is the first Trustee among ...
1,1,The Lost Hero,fantasy,"As the book opens, Jason awakens on a school ..."
2,2,The Eyes of the Overworld,fantasy,Cugel is easily persuaded by the merchant Fia...
3,3,Magic's Promise,fantasy,The book opens with Herald-Mage Vanyel return...
4,4,Taran Wanderer,fantasy,Taran and Gurgi have returned to Caer Dallben...


In [ ]:
def book_soup(x):
    return  x["title"] + " " + " ".join(x['genre']) + " "+x['summary']

books['summary'] = books['summary'].fillna('')
books['title'] = books['title'].fillna('')
books['soup'] = books.apply(book_soup, axis=1)

In [ ]:
books.loc[books['title']=="The Lost Hero",'soup'].values

array(['The Lost Hero f a n t a s y  As the book opens, Jason awakens on a school bus, unable to remember who or where he is, or anything about his past. He is sitting next to Piper McLean and Leo Valdez, who call him by name and say they are his girlfriend and best friend respectively. All three are part of a class field trip to the Grand Canyon, and after they arrive, a classmate Dylan turns into a Venti (Storm Spirit) and attacks the trio and their trip leader, Coach Gleeson Hedge. In the ensuing fight, Jason surprises everyone, including himself, when one of his coins turns into a sword which he uses to battle the storm spirits. Coach Hedge, who reveals himself to be a satyr during the fight, is taken captive by a fleeing spirit. After the battle, a flying chariot arrives to rescue the trio, but one of the people in it, Annabeth, is upset when she discovers that her missing boyfriend, Percy Jackson, is not there as she expected. Annabeth, seeking Percy, was told in a vision from th

In [ ]:
books2=books.iloc[:200]
books2

,index,title,genre,summary,soup
0,0,Drowned Wednesday,fantasy,Drowned Wednesday is the first Trustee among ...,Drowned Wednesday f a n t a s y Drowned Wedne...
1,1,The Lost Hero,fantasy,"As the book opens, Jason awakens on a school ...",The Lost Hero f a n t a s y As the book opens...
2,2,The Eyes of the Overworld,fantasy,Cugel is easily persuaded by the merchant Fia...,The Eyes of the Overworld f a n t a s y Cugel...
3,3,Magic's Promise,fantasy,The book opens with Herald-Mage Vanyel return...,Magic's Promise f a n t a s y The book opens ...
4,4,Taran Wanderer,fantasy,Taran and Gurgi have returned to Caer Dallben...,Taran Wanderer f a n t a s y Taran and Gurgi ...
...,...,...,...,...,...
195,195,Daja's Book,fantasy,The story opens with the four child-mages and...,Daja's Book f a n t a s y The story opens wit...
196,196,Space Cadet,fantasy,"In 2075, teenager Matt Dodson applies to join...","Space Cadet f a n t a s y In 2075, teenager M..."
197,197,Crown Duel,fantasy,Young Countess Meliara swears to her dying fa...,Crown Duel f a n t a s y Young Countess Melia...
198,198,Chocolat,fantasy,"The story begins as two strangers, Vianne Roc...",Chocolat f a n t a s y The story begins as tw...


In [ ]:
booksoup= books['soup']

In [ ]:
count = CountVectorizer(stop_words = "english")
count.fit(booksoup)

CountVectorizer(stop_words='english')

In [ ]:
books_matrix = count.transform(books2['soup'])

books_matrix.shape

(200, 55589)

In [ ]:
cosine_sim_books = cosine_similarity(books_matrix, books_matrix)

print(cosine_sim_books.shape)
print(cosine_sim_books)

(200, 200)
[[1.         0.04652033 0.0116711  ... 0.005679   0.02824729 0.01894442]
 [0.04652033 1.         0.02928447 ... 0.04122157 0.03290696 0.01833465]
 [0.0116711  0.02928447 1.         ... 0.01251222 0.03457539 0.00834784]
 ...
 [0.005679   0.04122157 0.01251222 ... 1.         0.03028305 0.07311503]
 [0.02824729 0.03290696 0.03457539 ... 0.03028305 1.         0.04040816]
 [0.01894442 0.01833465 0.00834784 ... 0.07311503 0.04040816 1.        ]]


In [ ]:
G2_w = nx.from_numpy_matrix(np.matrix(cosine_sim_books), create_using=nx.Graph)
layout = nx.spring_layout(G2_w)
# plt.figure(figsize=(16,10))

# nx.draw(G2_w, layout)
# labels = nx.get_edge_attributes(G2_w, "weight")
# nx.draw_networkx_edge_labels(G2_w, pos=layout, edge_labels=labels)
# plt.show()

In [ ]:
for i in range(200):
  for j in range(200):
    if (cosine_sim_books[i][j]>=0.05):
      cosine_sim_books[i][j]=1
    else :
      cosine_sim_books[i][j]=0
print(cosine_sim_books)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]]


In [ ]:
import pandas as pd
import networkx as nx
df = pd.DataFrame(cosine_sim_books)
G2_uw = nx.from_pandas_adjacency(df)
print(G2_uw)

Graph with 200 nodes and 3730 edges


In [ ]:
# plt.figure(figsize=(16,10))
# nx.draw(G2_uw, node_size=2500, with_labels=True, pos=nx.kamada_kawai_layout(G))

Combined- Movies and Books

In [ ]:
cosine_sim = cosine_similarity(cosine_sim_movies, cosine_sim_books)

print(cosine_sim.shape)
print(cosine_sim)

(200, 200)
[[0.08728716 0.12869789 0.         ... 0.19304684 0.13012001 0.17817416]
 [0.13416408 0.19781414 0.09128709 ... 0.13187609 0.13333333 0.12171612]
 [0.17693035 0.17391304 0.12038585 ... 0.19565217 0.17583479 0.24077171]
 ...
 [0.12649111 0.16318834 0.06454972 ... 0.16318834 0.18856181 0.23668232]
 [0.10776318 0.26481357 0.07332356 ... 0.1853695  0.16064387 0.24441186]
 [0.19547111 0.28820643 0.09975093 ... 0.19814192 0.25496724 0.31587796]]


In [ ]:
G3_w = nx.from_numpy_matrix(np.matrix(cosine_sim), create_using=nx.Graph)
layout = nx.spring_layout(G3_w)
print(G3_w)

Graph with 200 nodes and 16726 edges


In [ ]:
for i in range(200):
  for j in range(200):
    if (cosine_sim[i][j]>=0.15):
      cosine_sim[i][j]=1
    else :
      cosine_sim[i][j]=0
print(cosine_sim)

[[0. 0. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 1. 1. 1.]
 ...
 [0. 1. 0. ... 1. 1. 1.]
 [0. 1. 0. ... 1. 1. 1.]
 [1. 1. 0. ... 1. 1. 1.]]


In [ ]:
import pandas as pd
import networkx as nx
df = pd.DataFrame(cosine_sim)
G3_uw = nx.from_pandas_adjacency(df)
print(G3_uw)

Graph with 200 nodes and 16726 edges


In [ ]:
# plt.figure(figsize=(16,10))
# nx.draw(G3_uw, node_size=2500, with_labels=True, pos=nx.kamada_kawai_layout(G))

Eigen Vector Centrality for both weighted and unweighted

In [ ]:
nx.eigenvector_centrality(G3_w,weight='weight')

{0: 0.06189437057784533,
 1: 0.07040988839384536,
 2: 0.05146233005832909,
 3: 0.07830936448172794,
 4: 0.06556127338955312,
 5: 0.07235377862833686,
 6: 0.0695374384744043,
 7: 0.07881904424629185,
 8: 0.07140714950746341,
 9: 0.08047017473269132,
 10: 0.07839724609575452,
 11: 0.07888989142553485,
 12: 0.06570570629058481,
 13: 0.07349835174459468,
 14: 0.07676175177148872,
 15: 0.07783291107633797,
 16: 0.07563809471997812,
 17: 0.07284671762898143,
 18: 0.036030123444448614,
 19: 0.06868799539470055,
 20: 0.07194741431735964,
 21: 0.07120290402739658,
 22: 0.08030207476771704,
 23: 0.07131182586180003,
 24: 0.08081108174823594,
 25: 0.054143040993932405,
 26: 0.07586759155328811,
 27: 0.07037776549899968,
 28: 0.03390976010907928,
 29: 0.07500023150628471,
 30: 0.07857361152253808,
 31: 0.06553662087473656,
 32: 0.07841163243834023,
 33: 0.07224563792186137,
 34: 0.05705736694087503,
 35: 0.08031743669516563,
 36: 0.06663980653918347,
 37: 0.07600844493029778,
 38: 0.08099440168616

In [ ]:
nx.eigenvector_centrality(G3_uw,weight=None)

{0: 0.06189437057784533,
 1: 0.07040988839384536,
 2: 0.05146233005832909,
 3: 0.07830936448172794,
 4: 0.06556127338955312,
 5: 0.07235377862833686,
 6: 0.0695374384744043,
 7: 0.07881904424629185,
 8: 0.07140714950746341,
 9: 0.08047017473269132,
 10: 0.07839724609575452,
 11: 0.07888989142553485,
 12: 0.06570570629058481,
 13: 0.07349835174459468,
 14: 0.07676175177148872,
 15: 0.07783291107633797,
 16: 0.07563809471997812,
 17: 0.07284671762898143,
 18: 0.036030123444448614,
 19: 0.06868799539470055,
 20: 0.07194741431735964,
 21: 0.07120290402739658,
 22: 0.08030207476771704,
 23: 0.07131182586180003,
 24: 0.08081108174823594,
 25: 0.054143040993932405,
 26: 0.07586759155328811,
 27: 0.07037776549899968,
 28: 0.03390976010907928,
 29: 0.07500023150628471,
 30: 0.07857361152253808,
 31: 0.06553662087473656,
 32: 0.07841163243834023,
 33: 0.07224563792186137,
 34: 0.05705736694087503,
 35: 0.08031743669516563,
 36: 0.06663980653918347,
 37: 0.07600844493029778,
 38: 0.08099440168616

Weighted Measures

**The average shortest path length** is the sum of path lengths d(u,v) between all pairs of nodes (assuming the length is zero if v is not reachable from v) normalized by n*(n-1) where n is the number of nodes in G.

In [ ]:
nx.average_shortest_path_length(G3_w,weight='weight')

1.1669346733668342

**The Wiener index** is a topological index of a molecular graph, defined as the sum of distances between all pairs of its vertices.

In [ ]:
nx.wiener_index(G3_w,weight="weight")

23222.0

**Barrycenter** is also known as the median. It gives the most central node/nodes in the graph.

In [ ]:
nx.barycenter(G3_w,weight="weight")

[57]

**Stoer–Wagner algorithm** is a recursive algorithm to solve the minimum cut problem in undirected weighted graphs with non-negative weights.

In [ ]:
nx.stoer_wagner(G3_w, weight='weight')

(17.25816055363019,
 ([55],
  [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,
   133,
   134,
   135,
 

Unweighted Measures

**Closeness centrality**
It determines how close the given node is to all the other nodes in the graph.
Closeness centrality of a node u is the reciprocal of the average shortest path distance to u over all n-1 reachable nodes.

In [ ]:
nx.closeness_centrality(G3_uw,distance=None)

{0: 0.7865612648221344,
 1: 0.8577586206896551,
 2: 0.7158273381294964,
 3: 0.943127962085308,
 4: 0.8122448979591836,
 5: 0.8883928571428571,
 6: 0.8468085106382979,
 7: 0.9521531100478469,
 8: 0.8652173913043478,
 9: 0.9802955665024631,
 10: 0.943127962085308,
 11: 0.9567307692307693,
 12: 0.8223140495867769,
 13: 0.8844444444444445,
 14: 0.9255813953488372,
 15: 0.9386792452830188,
 16: 0.908675799086758,
 17: 0.8844444444444445,
 18: 0.6337579617834395,
 19: 0.8432203389830508,
 20: 0.8728070175438597,
 21: 0.8577586206896551,
 22: 0.9754901960784313,
 23: 0.8652173913043478,
 24: 0.9851485148514851,
 25: 0.7316176470588235,
 26: 0.9128440366972477,
 27: 0.8614718614718615,
 28: 0.6257861635220126,
 29: 0.908675799086758,
 30: 0.9521531100478469,
 31: 0.8122448979591836,
 32: 0.9476190476190476,
 33: 0.8766519823788547,
 34: 0.7537878787878788,
 35: 0.9754901960784313,
 36: 0.8326359832635983,
 37: 0.9128440366972477,
 38: 0.9851485148514851,
 39: 0.8326359832635983,
 40: 0.7803921

**Betweenness centrality** of a node u is the sum of the fraction of all-pairs shortest paths that pass through u.
It determines centrality based on how many shortest paths the given node is part of.


In [ ]:
nx.betweenness_centrality(G3_uw,weight=None)

{0: 0.00039066005879828344,
 1: 0.0006917508646017417,
 2: 0.0002653370911812861,
 3: 0.0010588792814388353,
 4: 0.00039746486030631625,
 5: 0.0011687245800375778,
 6: 0.0005600877431999839,
 7: 0.0011909763960822257,
 8: 0.0006552029596371594,
 9: 0.0016444461027265342,
 10: 0.0010239190302267487,
 11: 0.0013119206172067836,
 12: 0.0006599869236885971,
 13: 0.0006862103015806414,
 14: 0.0011148872761175,
 15: 0.001133848991632327,
 16: 0.0008225563420360966,
 17: 0.0008663021787690533,
 18: 0.00016423742084139157,
 19: 0.0006743628008393254,
 20: 0.000758765799374441,
 21: 0.0005173039130269907,
 22: 0.0014080469827101067,
 23: 0.000713134566262412,
 24: 0.0015328009152995906,
 25: 0.000396288609048245,
 26: 0.0010165413527553362,
 27: 0.0006972754159380081,
 28: 0.00015985169507299644,
 29: 0.000868648311632397,
 30: 0.0012712091064894457,
 31: 0.0004755458102041064,
 32: 0.0011557946833359264,
 33: 0.0007821927094600489,
 34: 0.00038522452545983703,
 35: 0.0015348270924739108,
 36: 

**Load centrality** takes into account all paths via a random walk method with weights assigned according to the path lengths. It is an alternative way of assessing the importance of the nodes.

In [ ]:
nx.load_centrality(G3_uw,weight=None)

{0: 0.00039066005879828306,
 1: 0.0006917508646017403,
 2: 0.0002653370911812857,
 3: 0.001058879281438835,
 4: 0.00039746486030631576,
 5: 0.001168724580037576,
 6: 0.0005600877431999833,
 7: 0.0011909763960822244,
 8: 0.0006552029596371585,
 9: 0.0016444461027265325,
 10: 0.0010239190302267478,
 11: 0.0013119206172067818,
 12: 0.0006599869236885965,
 13: 0.0006862103015806406,
 14: 0.0011148872761174982,
 15: 0.0011338489916323264,
 16: 0.0008225563420360961,
 17: 0.000866302178769052,
 18: 0.0001642374208413914,
 19: 0.0006743628008393248,
 20: 0.0007587657993744401,
 21: 0.0005173039130269898,
 22: 0.0014080469827101063,
 23: 0.0007131345662624111,
 24: 0.0015328009152995898,
 25: 0.0003962886090482444,
 26: 0.0010165413527553349,
 27: 0.0006972754159380073,
 28: 0.00015985169507299628,
 29: 0.000868648311632396,
 30: 0.0012712091064894444,
 31: 0.0004755458102041053,
 32: 0.0011557946833359249,
 33: 0.0007821927094600479,
 34: 0.00038522452545983665,
 35: 0.0015348270924739097,
 3

**Harmonic centrality** sums the inverses of the distances of a node to all the other nodes. It helps us find the nodes that can communicate or have an 'influence' over the rest of the network the best. We infer that harry potter can communicate and reach the other nodes the quickest. Nodes with higher harmonic centrality are said to be well connected or seem to have more influence/connections.

In [ ]:
nx.harmonic_centrality(G3_uw,distance=None)

{0: 172.0,
 1: 182.5,
 2: 159.5,
 3: 193.0,
 4: 176.0,
 5: 186.5,
 6: 181.0,
 7: 194.0,
 8: 183.5,
 9: 197.0,
 10: 193.0,
 11: 194.5,
 12: 177.5,
 13: 186.0,
 14: 191.0,
 15: 192.5,
 16: 189.0,
 17: 186.0,
 18: 141.5,
 19: 180.5,
 20: 184.5,
 21: 182.5,
 22: 196.5,
 23: 183.5,
 24: 197.5,
 25: 162.5,
 26: 189.5,
 27: 183.0,
 28: 139.5,
 29: 189.0,
 30: 194.0,
 31: 176.0,
 32: 193.5,
 33: 185.0,
 34: 166.5,
 35: 196.5,
 36: 179.0,
 37: 189.5,
 38: 197.5,
 39: 179.0,
 40: 171.0,
 41: 178.0,
 42: 191.0,
 43: 189.0,
 44: 191.0,
 45: 194.0,
 46: 197.0,
 47: 186.5,
 48: 191.0,
 49: 185.0,
 50: 196.0,
 51: 188.0,
 52: 184.0,
 53: 161.0,
 54: 138.5,
 55: 119.0,
 56: 198.0,
 57: 198.5,
 58: 174.5,
 59: 184.5,
 60: 195.5,
 61: 184.5,
 62: 180.0,
 63: 195.5,
 64: 158.0,
 65: 165.5,
 66: 165.0,
 67: 181.5,
 68: 168.5,
 69: 191.5,
 70: 188.5,
 71: 185.5,
 72: 177.0,
 73: 182.5,
 74: 160.5,
 75: 182.0,
 76: 184.5,
 77: 196.5,
 78: 143.5,
 79: 195.0,
 80: 187.0,
 81: 175.0,
 82: 195.0,
 83: 190.5,
 8

In [ ]:
nx.write_edgelist(G3_uw, '/content/graph_data.csv', comments='#', delimiter=',', data=True, encoding='utf-8')